In [ ]:
import ee

!pip install zarr
!pip install xarray
!pip install xarray[complete]
!pip install minio
!pip install nest-asyncio
import zarr
import xarray as xr
import pandas as pd
import numpy as np
import io
import minio
import os
import asyncio

import nest_asyncio
nest_asyncio.apply()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.0/187.0 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 53.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.7/53.7 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.8/48.8 kB 1.4 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of dask-expr to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.8/72.8 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 51.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.4/117.4 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 33.4 MB/s eta 0

In [ ]:
ee.Authenticate()
ee.Initialize(project='planar-osprey-377213')

In [ ]:
# Open the Zarr dataset
# ds = xr.open_zarr("gs://gcp-public-data-arco-era5/ar/1959-2022-6h-1440x721.zarr")
ds = xr.open_zarr("gs://gcp-public-data-arco-era5/ar/1959-2022-full_37-6h-0p25deg_derived.zarr")

# Define filtering parameters
start_year = "2019-01-01"
end_year = "2019-01-10"
selected_levels = [50, 100, 150, 200, 250, 300, 400, 500, 600, 700, 850, 925, 1000]

# Filter dataset
resampled_ds = ds.sel(
    time=slice(start_year, end_year),  # Filter by time range
    level=selected_levels  # Filter by pressure levels
)

resampled_ds = resampled_ds.where((resampled_ds.time.dt.hour == 0) | (resampled_ds.time.dt.hour == 6), drop=True)

# Resample to 128x256 spatial resolution
# resampled_ds = resampled_ds.interp(
#     latitude=pd.Series(np.linspace(resampled_ds.latitude.min(), resampled_ds.latitude.max(), 128)),
#     longitude=pd.Series(np.linspace(resampled_ds.longitude.min(), resampled_ds.longitude.max(), 256)),
#     method='nearest'
# )

# Ensure that specific variables are included
resampled_ds = resampled_ds[['10m_u_component_of_wind', '10m_v_component_of_wind',
                            '2m_temperature', 'mean_sea_level_pressure',
                            'geopotential', 'specific_humidity',
                            'u_component_of_wind', 'v_component_of_wind',
                            'temperature']]
resampled_ds

<xarray.Dataset> Size: 6GB
Dimensions:                  (time: 20, latitude: 721, longitude: 1440,
                              level: 13)
Coordinates:
  * latitude                 (latitude) float32 3kB 90.0 89.75 ... -89.75 -90.0
  * level                    (level) int64 104B 50 100 150 200 ... 850 925 1000
  * longitude                (longitude) float32 6kB 0.0 0.25 ... 359.5 359.8
  * time                     (time) datetime64[ns] 160B 2019-01-01 ... 2019-0...
Data variables:
    10m_u_component_of_wind  (time, latitude, longitude) float32 83MB dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>
    10m_v_component_of_wind  (time, latitude, longitude) float32 83MB dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>
    2m_temperature           (time, latitude, longitude) float32 83MB dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>
    mean_sea_level_pressure  (time, latitude, longitude) float32 83MB dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>
    geopotential             (time, level, latitude, longitude) float32 1GB dask.array<chunksize=(1, 13, 721, 1440), meta=np.ndarray>
    specific_humidity        (time, level, latitude, longitude) float32 1GB dask.array<chunksize=(1, 13, 721, 1440), meta=np.ndarray>
    u_component_of_wind      (time, level, latitude, longitude) float32 1GB dask.array<chunksize=(1, 13, 721, 1440), meta=np.ndarray>
    v_component_of_wind      (time, level, latitude, longitude) float32 1GB dask.array<chunksize=(1, 13, 721, 1440), meta=np.ndarray>
    temperature              (time, level, latitude, longitude) float32 1GB dask.array<chunksize=(1, 13, 721, 1440), meta=np.ndarray>

In [ ]:
def saveFileLocal(file_path, data):
    new_file_path = os.path.join("/content/data", file_path)
    dir = os.path.dirname(new_file_path)
    if not os.path.exists(dir):
        os.makedirs(dir)

    np.save(new_file_path, data)

In [ ]:
single_level_vnames = ["u10", "v10", "t2m", "msl"]
multi_level_vnames = ["z", "q", "u", "v", "t"]
height_level = [50, 100, 150, 200, 250, 300, 400, 500, 600, 700, 850, 925, 1000]

single_level_mapping = {
    "10m_u_component_of_wind": "u10",
    "10m_v_component_of_wind": "v10",
    "2m_temperature": "t2m",
    "mean_sea_level_pressure": "msl",
}

multi_level_mapping = {
    "geopotential": "z",
    "specific_humidity": "q",
    "u_component_of_wind": "u",
    "v_component_of_wind": "v",
    "temperature": "t"
}



def uploadMultiLevel(time):
    time_value = str(resampled_ds.isel(time=time).time.values)
    timestamp = str(time_value).split(".")[0]
    year = pd.Timestamp(time_value).year

    def uploadKey(vname):
        file = os.path.join(str(year), timestamp).replace("T", "/")

        def uploadLevel(levelIdx):
            height = height_level[levelIdx]
            data = resampled_ds.isel(time=time)[vname][levelIdx]

            mapped_vname = multi_level_mapping[vname]

            url = f"{file}-{mapped_vname}-{height}.0.npy"

            # uploadFileToS3("era-bucket", url, data.values)
            saveFileLocal(url, data.values)
            print(f"Upload file {url}")

        for i in range(13):
          uploadLevel(i)

    for vname in multi_level_mapping.keys():
        uploadKey(vname)


def uploadSingleLevel(time):
    time_value = str(resampled_ds.isel(time=time).time.values)
    timestamp = str(time_value).split(".")[0]
    year = pd.Timestamp(time_value).year

    def uploadKey(vname):
        data = resampled_ds.isel(time=time)[vname]

        mapped_vname = single_level_mapping[vname]

        file = os.path.join("single/", str(year), timestamp).replace("T", "/")
        url = f"{file}-{mapped_vname}.npy"

        # uploadFileToS3("era-bucket", url, data.values)
        saveFileLocal(url, data.values)

        print(f"Upload file {url}")

    for vname in single_level_mapping.keys():
        uploadKey(vname)

def uploadAll(time):
    uploadMultiLevel(time)
    uploadSingleLevel(time)

In [ ]:
def main():
    # Run all uploads concurrently
    for i in range(len(resampled_ds["time"])):
        uploadAll(i)

main()

Upload file 2019/2019-01-01/00:00:00-z-50.0.npy
Upload file 2019/2019-01-01/00:00:00-z-100.0.npy
Upload file 2019/2019-01-01/00:00:00-z-150.0.npy
Upload file 2019/2019-01-01/00:00:00-z-200.0.npy
Upload file 2019/2019-01-01/00:00:00-z-250.0.npy
Upload file 2019/2019-01-01/00:00:00-z-300.0.npy
Upload file 2019/2019-01-01/00:00:00-z-400.0.npy
Upload file 2019/2019-01-01/00:00:00-z-500.0.npy
Upload file 2019/2019-01-01/00:00:00-z-600.0.npy
Upload file 2019/2019-01-01/00:00:00-z-700.0.npy
Upload file 2019/2019-01-01/00:00:00-z-850.0.npy
Upload file 2019/2019-01-01/00:00:00-z-925.0.npy
Upload file 2019/2019-01-01/00:00:00-z-1000.0.npy
Upload file 2019/2019-01-01/00:00:00-q-50.0.npy
Upload file 2019/2019-01-01/00:00:00-q-100.0.npy
Upload file 2019/2019-01-01/00:00:00-q-150.0.npy
Upload file 2019/2019-01-01/00:00:00-q-200.0.npy
Upload file 2019/2019-01-01/00:00:00-q-250.0.npy
Upload file 2019/2019-01-01/00:00:00-q-300.0.npy
Upload file 2019/2019-01-01/00:00:00-q-400.0.npy
Upload file 2019/2019

In [ ]:
# Zip folder
!zip -r "/content/file.zip" "/content/data/"

  adding: content/data/ (stored 0%)
  adding: content/data/2019/ (stored 0%)
  adding: content/data/2019/2019-01-04/ (stored 0%)
  adding: content/data/2019/2019-01-04/06:00:00-q-300.0.npy (deflated 40%)
  adding: content/data/2019/2019-01-04/06:00:00-u-850.0.npy (deflated 29%)
  adding: content/data/2019/2019-01-04/06:00:00-z-1000.0.npy (deflated 30%)
  adding: content/data/2019/2019-01-04/00:00:00-q-850.0.npy (deflated 34%)
  adding: content/data/2019/2019-01-04/00:00:00-v-700.0.npy (deflated 28%)
  adding: content/data/2019/2019-01-04/00:00:00-v-200.0.npy (deflated 22%)
  adding: content/data/2019/2019-01-04/00:00:00-q-925.0.npy (deflated 35%)
  adding: content/data/2019/2019-01-04/00:00:00-v-925.0.npy (deflated 30%)
  adding: content/data/2019/2019-01-04/06:00:00-z-500.0.npy (deflated 41%)
  adding: content/data/2019/2019-01-04/06:00:00-v-400.0.npy (deflated 23%)
  adding: content/data/2019/2019-01-04/06:00:00-v-100.0.npy (deflated 26%)
  adding: content/data/2019/2019-01-04/00:00:

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

Mounted at /content/gdrive


In [ ]:
!cp "/content/file.zip" '/content/gdrive/My Drive/file.zip'